In [6]:
# Importing modules
import pandas as pd
import os
import re
os.chdir('..')

In [7]:
#loading the user review data
userreviews = pd.read_csv(r'C:\Users\koenm\OneDrive\Bureaublad\Thesis\Metacritic_thesis\4 Topic_modeling\Topic modelling input\usercleanforlda.csv', delimiter=";")

#print head
userreviews.head()

,url,user_review_body
0,https://www.metacritic.com/music/the-order-of-...,album amazing one bests albuns without doubtt...
1,https://www.metacritic.com/music/the-order-of-...,excellent album enough earworms throughout war...
2,https://www.metacritic.com/music/the-order-of-...,checked due comparisons norah jones whose musi...
3,https://www.metacritic.com/music/the-order-of-...,soulbluesrock album enough maintain solidity c...
4,https://www.metacritic.com/music/the-order-of-...,decided listen album account metascore simply ...


In [3]:
#Prepare data for LDA Analysis because it has already been preprocessed, therefor no cleaning is needed

In [8]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\koenm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = userreviews.user_review_body.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['album', 'amazing', 'one', 'bests', 'albuns', 'without', 'doubtthe', 'lycris', 'awsome', 'album', 'reaaly', 'good', 'congrats', 'incredible', 'albun']


In [10]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]


In [18]:
from pprint import pprint

# number of topics
num_topics = 6

In [19]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

In [20]:
# Print the Keyword in the 20 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.025*"album" + 0.021*"review" + 0.020*"contains" + 0.020*"view" + '
  '0.019*"click" + 0.019*"expand" + 0.019*"spoilers" + 0.011*"best" + '
  '0.008*"one" + 0.007*"song"'),
 (1,
  '0.059*"album" + 0.014*"best" + 0.013*"songs" + 0.013*"good" + 0.012*"great" '
  '+ 0.010*"one" + 0.010*"like" + 0.009*"amazing" + 0.008*"music" + '
  '0.008*"really"'),
 (2,
  '0.033*"album" + 0.025*"de" + 0.023*"que" + 0.015*"um" + 0.011*"com" + '
  '0.011*"uma" + 0.010*"da" + 0.010*"nao" + 0.009*"em" + 0.007*"mais"'),
 (3,
  '0.032*"album" + 0.015*"like" + 0.012*"songs" + 0.009*"one" + 0.008*"song" + '
  '0.006*"music" + 0.006*"good" + 0.006*"really" + 0.005*"love" + '
  '0.005*"band"'),
 (4,
  '0.032*"album" + 0.022*"love" + 0.018*"best" + 0.008*"music" + '
  '0.008*"record" + 0.007*"one" + 0.006*"pop" + 0.006*"new" + 0.006*"like" + '
  '0.005*"year"'),
 (5,
  '0.033*"de" + 0.027*"album" + 0.024*"que" + 0.018*"es" + 0.017*"la" + '
  '0.014*"el" + 0.013*"un" + 0.012*"en" + 0.008*"con" + 0.008*"una

In [10]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

In [11]:
# 6. What is the Dominant topic and its percentage contribution in each document

C:\Users\koenm\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=userreviews['user_review_body']):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if lda_model.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = lda_model.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

C:\Users\koenm\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyboardInterrupt: 

In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=userreviews['user_review_body'])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)